In [ ]:
#Imports 
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import time
import torch
import matplotlib.pyplot as plt 
import numpy as np

from torch import nn, optim
from torchvision import datasets
from torchvision import models
from torchvision import transforms
from PIL import Image
from collections import OrderedDict
import torch.nn.functional as F



In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
#Loading Data using torch vision
data_transforms = transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                             ])

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])

valid_transforms = transforms.Compose([transforms.Resize(255),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

test_transforms = transforms.Compose([transforms.Resize(255),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
# Loading datasets with ImageFolder
data_data_sets = datasets.ImageFolder(data_dir, transform=data_transforms)
train_data_set = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data_set = datasets.ImageFolder(valid_dir, transform=valid_transforms)
test_data_set = datasets.ImageFolder(test_dir, transform=test_transforms)

#Using the image datasets and the trainforms
data_loaders = torch.utils.data.DataLoader(data_data_sets, batch_size = 32, shuffle = True)
train_loaders = torch.utils.data.DataLoader(train_data_set, batch_size = 32, shuffle = True)
valid_loaders = torch.utils.data.DataLoader(valid_data_set, batch_size = 32, shuffle = True)
test_loaders = torch.utils.data.DataLoader(test_data_set, batch_size = 32, shuffle = True)

In [ ]:
#Label mapping using JSON
#will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [ ]:
#Building a Network
model = models.vgg19(pretrained=True)
print(model)

for param in model.parameters():
    param.requires_grad = False
    
    
classifier = nn.Sequential(OrderedDict([
                                        ('fc1', nn.Linear(25088, 4096)),
                                        ('relu', nn.ReLU()),
                                        ('fc2', nn.Linear(4096, 102)),
                                        ('fc3', nn.LogSoftmax(dim=1))
                                        ]))

model.classifier = classifier

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)


model


In [ ]:
#Training the Network
def train_model(model, trainloader, validloader, epochs, print_every, criterion, optimizer, device = torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    epochs = epochs
    print_every = print_every
    steps = 0

    # change to cuda
    model.to(device)

    for e in range(epochs):
        running_loss = 0
        for ii, (inputs, labels) in enumerate(trainloader):
            steps += 1

            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward and backward passes
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            
            #Calculating Loss
            running_loss += loss.item()

            if steps % print_every == 0:
                valid_accuracy = check_validation_set(validloader,device)
                print("Epoch: {}/{}... ".format(e+1, epochs),
                      "Loss: {:.4f}".format(running_loss/print_every),
                      "Validation Accuracy: {}".format(round(valid_accuracy,4)))

                running_loss = 0
    
    
def check_validation_set(valid_loader,device = torch.device("cuda" if torch.cuda.is_available() else "cpu")):    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valid_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total 



train_model(model, train_loaders, valid_loaders, 3, 40, criterion, optimizer)


In [ ]:
#Testing the Network
def check_accuracy_on_test(test_loaders,device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loaders:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    return correct / total 


In [ ]:
check_accuracy_on_test(test_loaders)

In [ ]:
#Saving checkpoint
model.class_to_idx = train_data_set.class_to_idx

checkpoint = {'transfer_model': 'vgg19',
              'input_size': 25088,
              'output_size': 102,
              'features': model.features,
              'classifier': model.classifier,
              'optimizer': optimizer.state_dict(),
              'state_dict': model.state_dict(),
              'idx_to_class': {v: k for k, v in train_data_set.class_to_idx.items()}
             }

torch.save(checkpoint, 'check.pth')

In [ ]:
#Loading up the Checkpoint
def load_checkpoint(path):
    model_information = torch.load(path)
    model = models.vgg19(pretrained=True)
    classifier = nn.Sequential(OrderedDict([
                              ('fc1', nn.Linear(25088, 4096)),
                              ('relu', nn.ReLU()),
                              ('fc2', nn.Linear(4096, 102)),
                              ('output', nn.LogSoftmax(dim=1))
                              ]))

    model.classifier = classifier
    model.load_state_dict(model_information['state_dict'])
    
    return model, model_information

model, model_information = load_checkpoint('check.pth')

In [ ]:
def process_image(image):

    
    #  Processing a PIL image for use in a PyTorch model
    adj= transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return adj(Image.open(image))

In [ ]:
#Method Used to CHECK by converting pytorch tensors and displayin them

def imshow(image, ax=None, title=None):
    
    if ax is None:
        fig, ax = plt.subplots()
    
   
    image = image.numpy().transpose((1, 2, 0))
    
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
#predicting
#Using "topk"to display the top 5 most probable classes
def predict(image_path, model, topk=5):
    
    model.to('cuda')
    img_torch = process_image(image_path)
    img_torch = img_torch.unsqueeze_(0).float()
    
    with torch.no_grad():
        output = model.forward(img_torch.cuda())
        
    probability = F.softmax(output.data,dim=1)
    
    return probability.topk(topk)

img = ("flowers/test/10/image_07104.jpg")
predict(img, model)